In [1]:
'''
Project Euler Problem 068: Magic 5-gon Ring
'''

'\nProject Euler Problem 068: Magic 5-gon Ring\n'

In [200]:
import sympy as sp
from itertools import permutations

In [372]:
def pick_outer(n, highest, sum, start=1):
    # print(start, n, sum)

    selection = []
    if n == 1:
        selection.append([] if sum > highest else [sum])
    for value in range(start, min((sum-(n-1)*(n)//2)//n+1, highest)):
        if n > 1:
            subsets = pick_outer(n-1, highest, sum-value, value+1)
            for subset in subsets:
                if len(subset) == n-1:
                    selection.append([value]+subset)

    return selection


def complement(xlist, n):
    return [x for x in range(1, n+1) if x not in xlist]

In [519]:
outer = sp.IndexedBase('o')
inner = sp.IndexedBase('i')
i = sp.symbols('i', cls=sp.Idx)
S = sp.symbols('S')

n = 5
nleg = 3

equations = [sp.Eq(outer[i]+sum([inner[(i+j)%n] for j in range(1, nleg)]), S) for i in range(n)]
outers = [outer[i] for i in range(n)]
inners = [inner[i] for i in range(n)]
solution = sp.solve(equations, inners)

In [521]:
f_inners = [sp.lambdify(outers, solution[k]) for k in solution]
solutions = []

for outer_sum in range(1, n*(2*n+1)+1):
    inner_sum = n*(2*n+1) - outer_sum
    leg_sum = (outer_sum + (nleg-1)*inner_sum)
    # print(outer_sum, inner_sum, leg_sum)

    if leg_sum % n == 0:
        leg_sum //= n
        for pick in pick_outer(n, 2*n, outer_sum):
            for perm in permutations(pick[1:]):
                perm = [pick[0]]+list(perm)
                if sorted(pick_inner := [int(f(*perm).subs(S, leg_sum)) for f in f_inners]) == complement(perm, 2*n):
                    sol = [[[perm[i%n]]+[pick_inner[(i+j)%n] for j in range(1, nleg)] for i in range(n)]]
                    solutions += sol

legstrings = [''] * len(solutions)
for i, sol in enumerate(solutions):
    for leg in sol:
        legstrings[i] += ''.join(map(str, leg))

print(f'There are {len(solutions)} solutions:')
print(solutions)

lengths = set([len(s) for s in legstrings])

for length in lengths:
    legvalues = [int(ls) for ls in legstrings if len(ls) == length]
    print(f'The maximum string with {length} digits has a value {max(legvalues)} ')


There are 12 solutions:
[[[1, 8, 10], [2, 10, 7], [3, 7, 9], [4, 9, 6], [5, 6, 8]], [[1, 10, 8], [5, 8, 6], [4, 6, 9], [3, 9, 7], [2, 7, 10]], [[1, 6, 10], [3, 10, 4], [5, 4, 8], [7, 8, 2], [9, 2, 6]], [[1, 10, 6], [9, 6, 2], [7, 2, 8], [5, 8, 4], [3, 4, 10]], [[2, 7, 8], [5, 8, 4], [3, 4, 10], [6, 10, 1], [9, 1, 7]], [[2, 8, 7], [9, 7, 1], [6, 1, 10], [3, 10, 4], [5, 4, 8]], [[2, 5, 9], [4, 9, 3], [6, 3, 7], [8, 7, 1], [10, 1, 5]], [[2, 9, 5], [10, 5, 1], [8, 1, 7], [6, 7, 3], [4, 3, 9]], [[2, 4, 10], [5, 10, 1], [8, 1, 7], [6, 7, 3], [9, 3, 4]], [[2, 10, 4], [9, 4, 3], [6, 3, 7], [8, 7, 1], [5, 1, 10]], [[6, 3, 5], [7, 5, 2], [8, 2, 4], [9, 4, 1], [10, 1, 3]], [[6, 5, 3], [10, 3, 1], [9, 1, 4], [8, 4, 2], [7, 2, 5]]]
The maximum string with 16 digits has a value 6531031914842725 
The maximum string with 17 digits has a value 28797161103104548 
